In [6]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.models import HoverTool, ColumnDataSource

# --- 1. Generate and Enhance Sample Data ---
dates = pd.date_range("2023-10-01", periods=14, freq="D")
base_temp = 15
amplitude = 5
real_temp = base_temp + amplitude * np.sin(np.arange(14) * 0.8) + np.random.normal(0, 0.5, 14)
prediction_error = np.random.normal(0, 1.2, 14)
predicted_temp = real_temp - prediction_error + np.random.normal(0, 0.4, 14)
df = pd.DataFrame({
    'date': dates,
    'real_temp': real_temp,
    'predicted_temp': predicted_temp
})

## --- MODIFIED: Calculate accuracy instead of ratio --- ##
# Calculate the absolute difference in temperature
df['difference_abs'] = (df['real_temp'] - df['predicted_temp']).abs()

# Calculate prediction accuracy as a percentage
# Accuracy = (1 - |real - predicted| / |real|) * 100
# Handle edge cases: division by zero and accuracy < 0
accuracy_calc = np.where(
    df['real_temp'] == 0,
    np.where(df['predicted_temp'] == 0, 100.0, 0.0), # If real is 0, accuracy is 100% only if prediction is also 0
    (1 - (df['difference_abs'] / df['real_temp'].abs())) * 100
)

# Add the new column to the DataFrame, clipping values below 0.
df['prediction_accuracy'] = np.clip(accuracy_calc, 0, 100)

# Create the source AFTER adding the new columns
source = ColumnDataSource(df)


# --- 2. Create Two Hover Tools ---
TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

# Hover tool for the LINES (remains the same)
line_hover = HoverTool(
    tooltips=[
        ("Date", "@date{%F}"),
        ("Real Temp", "@real_temp{0.2f} °C"),
        ("Predicted Temp", "@predicted_temp{0.2f} °C")
    ],
    formatters={'@date': 'datetime'},
    mode='vline'
)

## MODIFIED: Hover tool for the SHADED AREA now shows accuracy ##
varea_hover = HoverTool(
    tooltips=[
        ("Date", "@date{%F}"),
        ("Prediction Accuracy", "@prediction_accuracy{0.1f}%"),
        ("Difference", "@difference_abs{0.2f} °C")
    ],
    formatters={'@date': 'datetime'},
    mode='vline'
)


# --- 3. Create Figure and Plot Glyphs ---
# This part remains the same, just putting the pieces together.
p = figure(
    height=450, sizing_mode="stretch_width",
    title="Real vs. Predicted Temperature with Accuracy Shading",
    x_axis_label="Date", y_axis_label="Temperature (°C)",
    x_axis_type="datetime",
    tools=[line_hover, varea_hover] + TOOLS.split(',')
)
p.title.align = 'center'
p.title.text_font_size = '1.5em'

band = p.varea(
    x='date', y1='real_temp', y2='predicted_temp', source=source,
    fill_color='mediumseagreen', fill_alpha=0.4, legend_label="Prediction Difference"
)
real_line = p.line(
    x='date', y='real_temp', source=source,
    legend_label="Real Temperature", color="dodgerblue", line_width=3
)
predicted_line = p.line(
    x='date', y='predicted_temp', source=source,
    legend_label="Predicted Temperature", color="tomato", line_width=3, line_dash="dashed"
)

# Assign each hover tool to its specific renderer(s)
line_hover.renderers = [real_line, predicted_line]
varea_hover.renderers = [band]


# --- 4. Customize and Generate ---
p.legend.location = "top_left"
p.legend.click_policy = "hide"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_alpha = 0.5
p.ygrid.grid_line_dash = [6, 4]

# I changed the shading color to green to better reflect "accuracy"
band.glyph.fill_color = "mediumseagreen"

output_file("temperature_accuracy_plot.html", title="Temperature Prediction Accuracy")
show(p)
print("Successfully generated 'temperature_accuracy_plot.html'")

Successfully generated 'temperature_accuracy_plot.html'
